In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.storage import LocalFileStore
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS

llm = ChatOpenAI(
    temperature=0.1
)

cache_dir = LocalFileStore("./cache")

splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100
)

loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

FileNotFoundError: [Errno 2] No such file or directory: './files/document.txt'

In [71]:
# memory 셋팅

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True , llm=llm, max_token_limit =20)
# return_messages=True 면 chat 형식이다 

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

def load_memory(_):
    return memory.load_memory_variables({})["history"]


In [73]:
# memory 셋팅 + 검색할 content 셋팅

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)



In [75]:

from langchain.schema.runnable import RunnablePassthrough

chain = (
    {
        "context" : retriver,
        "question": RunnablePassthrough(),
        "history": load_memory,
    }
    | prompt
    | llm
)


In [76]:
def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context({"input": question}, {"output": result.content})
    print(result.content)

In [79]:
invoke_chain("Is Aaronson guilty?")

Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.


In [80]:
invoke_chain("What message did he write in the table?")

I'm sorry, I don't have that information.


In [81]:
invoke_chain("Who is Julia?")

Julia is a significant person in the text, with whom the main character has a deep connection and conflicting emotions.


In [68]:
invoke_chain("Aaronson 는 테이블에 어떤 메시지를 썼나요?")

죄송합니다, 그 정보는 제가 가지고 있지 않습니다.


In [69]:
memory.load_memory_variables({})["history"]

[HumanMessage(content='Aaronson 은 유죄인가요?'),
 AIMessage(content='네, Jones, Aaronson, 그리고 Rutherford는 그들이 기소된 범죄로 유죄로 판결받았습니다.'),
 HumanMessage(content='그가 테이블에 어떤 메시지를 썼나요?'),
 AIMessage(content='죄송합니다, 그 정보는 제가 가지고 있지 않습니다.'),
 HumanMessage(content='Julia 는 누구인가요?'),
 AIMessage(content='Julia는 주인공과 관련된 인물로, 그의 사랑이자 도움이 필요한 사람입니다.'),
 HumanMessage(content='Aaronson 는 테이블에 어떤 메시지를 썼나요?'),
 AIMessage(content='죄송합니다, 그 정보는 제가 가지고 있지 않습니다.')]